# Tidy Data

In [2]:
# AWS S3 bucket path prefix - update with your own VAULT_NAME
VAULT_NAME = 'cstephenson'

S3_PREFIX = 's3a://%s:%s@' % (dbutils.secrets.get(VAULT_NAME, 'access_key'), dbutils.secrets.get(VAULT_NAME, 'secret_key').replace('/', '%2F'))
print(S3_PREFIX)

s3a://[REDACTED]:[REDACTED]@

## What is Tidy Data?

A paper by __Hadley Wickham__ (http://vita.had.co.nz/papers/tidy-data.pdf) describes Tidy Data as a way to structure the messy and disorganised data into data that will faciliate your analysis activities.  

Often we recieve data in different formats and structures from clients. Tidy Data is about 'structuring' the data in a conistent manner. Naturally before 'tidying' data we should ensure we can load the data into a dataframe correctly and fix datatype issues - these activities are covered in other workshops  

This notebook will illustrate some common patterns to demonstrate "tidying" your data using Spark SQL

The definition of Tidy Data, according to Wickam are:  
  
* Each variable forms a column and contains values
* Each observation forms a row
* Each type of observational unit forms a table

__*Where*__  
  
__Variable:__ A measurement or an attribute. e.g. Height, weight, sex, etc.  
__Value:__ The actual measurement or attribute. e.g. 152 cm, 80 kg, female, etc.  
__Observation:__ All values measure on the same unit. e.g. Each person.  

*__Note__ Those of you familiar with Boyce-Codd "Normal Form" for modeling relational data may spot some simalarities to Tidy Data*

In [4]:
df_untidy = spark.read.format('csv').option('header', 'true').option('inferSchema','true').option('delimiter',',').load(S3_PREFIX + 'sagacity-analyst-scratch/workshops/tidy_data/data/untidy.csv')
display(df_untidy.orderBy('name'))

name,treatment_A,treatment_B
Jane Doe,16,11
John Smith,null,2
Mary Johnson,3,1


In [5]:
df_tidy = spark.read.format('csv').option('header', 'true').option('inferSchema','true').option('delimiter',',').load(S3_PREFIX + 'sagacity-analyst-scratch/workshops/tidy_data/data/tidy.csv')
display(df_tidy.orderBy('name','treatment'))

name,treatment,result
Jane Doe,A,16
Jane Doe,B,11
John Smith,A,null
John Smith,B,2
Mary Johnson,A,3
Mary Johnson,B,1


Examples of __untidy__ datasets we will look at:

1) Column headers are values, not variable names. 
  
2) Multiple variables are stored in one column. 
  
3) Variables are stored in both rows and columns.  
  
4) Multiple types of observational units are stored in the same table.  
  
5) A single observational unit is stored in multiple tables.

## 1) Column headers are values, not variable names
Pew Research Center Dataset  
This dataset explores the relationship between income and religion.  

__Problems:__  
* The columns headers are composed of the possible income values.

In [8]:
df_pew = spark.read.format('csv'
  ).option('header', 'true'
  ).option('inferSchema','true'
  ).option('delimiter',','
  ).load(S3_PREFIX + 'sagacity-analyst-scratch/workshops/tidy_data/data/pew-raw.csv')

df_pew.createOrReplaceTempView('pew_raw')
display(df_pew)

religion,0_10k,10_20k,20_30k,30_40k,40_50k,50_75k
Agnostic,27,34,60,81,76,137
Atheist,12,27,37,52,35,70
Buddhist,27,21,30,34,33,58
Catholic,418,617,732,670,638,1116
Dont know/refused,15,14,15,11,10,35
Evangelical Prot,575,869,1064,982,881,1486
Hindu,1,9,7,9,11,34
Historically Black Prot,228,244,236,238,197,223
Jehovahs Witness,20,27,24,24,21,30
Jewish,19,19,25,25,30,95


To un-pivot (transform columns into rows) the data, we can use the Spark SQL STACK statement

```STACK([no_cols], [row_value1], [column_name1], ... , [row_valueN], [column_nameN])``` 

https://spark.apache.org/docs/latest/api/sql/#stack

In [10]:
%sql
SELECT 
  religion
, STACK(6
  , '0-10k', 0_10k
  , '10-20k', 10_20k
  , '20-30k', 20_30k
  , '30-40k', 30_40k
  , '40-50k', 40_50k
  , '50-75k', 50_75k
  ) AS (
    salary_band, salary_value
  ) 
FROM 
  pew_raw
ORDER BY 
  religion
, salary_band

-- , '10-20k', 10-20k, '20-30k', 20-30k, '30-40k', 30-40k, '40-50k', 40-50k, '50-75k', 40-50k

religion,salary_band,salary_value
Agnostic,0-10k,27
Agnostic,10-20k,34
Agnostic,20-30k,60
Agnostic,30-40k,81
Agnostic,40-50k,76
Agnostic,50-75k,137
Atheist,0-10k,12
Atheist,10-20k,27
Atheist,20-30k,37
Atheist,30-40k,52


Billboard Top 100 Dataset  
This dataset represents the weekly rank of songs from the moment they enter the Billboard Top 100 to the subsequent 75 weeks.

__Problems:__
* The columns headers are composed of values: the week number (x1st_week, …)
* If a song is in the Top 100 for less than 75 weeks, the remaining columns are filled with missing values.

In [12]:
df_billboard = spark.read.format('csv'
  ).option('header', 'true'
  ).option('inferSchema','true'
  ).option('delimiter',','
  ).load(S3_PREFIX + 'sagacity-analyst-scratch/workshops/tidy_data/data/billboard.csv')


df_billboard.createOrReplaceTempView('billboard')
display(df_billboard)

year,artist_inverted,track,time,genre,date_entered,date_peaked,x1st_week,x2nd_week,x3rd_week,x4th_week,x5th_week,x6th_week,x7th_week,x8th_week,x9th_week,x10th_week,x11th_week,x12th_week,x13th_week,x14th_week,x15th_week,x16th_week,x17th_week,x18th_week,x19th_week,x20th_week,x21st_week,x22nd_week,x23rd_week,x24th_week,x25th_week,x26th_week,x27th_week,x28th_week,x29th_week,x30th_week,x31st_week,x32nd_week,x33rd_week,x34th_week,x35th_week,x36th_week,x37th_week,x38th_week,x39th_week,x40th_week,x41st_week,x42nd_week,x43rd_week,x44th_week,x45th_week,x46th_week,x47th_week,x48th_week,x49th_week,x50th_week,x51st_week,x52nd_week,x53rd_week,x54th_week,x55th_week,x56th_week,x57th_week,x58th_week,x59th_week,x60th_week,x61st_week,x62nd_week,x63rd_week,x64th_week,x65th_week,x66th_week,x67th_week,x68th_week,x69th_week,x70th_week,x71st_week,x72nd_week,x73rd_week,x74th_week,x75th_week,x76th_week
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,78,63,49,33,23,15,7,5,1,1,1,1,1,1,1,1,1,1,1,2,3,7,10,12,15,22,29,31,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12T00:00:00.000+0000,2000-04-08T00:00:00.000+0000,15,8,6,5,2,3,2,2,1,1,1,1,1,1,1,1,1,1,8,15,19,21,26,36,48,47,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23T00:00:00.000+0000,2000-01-29T00:00:00.000+0000,71,48,43,31,20,13,7,6,4,4,4,6,4,2,1,1,1,2,1,2,4,8,8,12,14,17,21,24,30,34,37,46,47,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2000,Madonna,Music,3:45,Rock,2000-08-12T00:00:00.000+0000,2000-09-16T00:00:00.000+0000,41,23,18,14,2,1,1,1,1,2,2,2,2,2,4,8,11,16,20,25,27,27,29,44,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05T00:00:00.000+0000,2000-10-14T00:00:00.000+0000,57,47,45,29,23,18,11,9,9,11,1,1,1,1,4,8,12,22,23,43,44,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2000,Janet,Doesn't Really Matter,4:17,Rock,2000-06-17T00:00:00.000+0000,2000-08-26T00:00:00.000+0000,59,52,43,30,29,22,15,10,10,5,1,1,1,2,2,3,3,7,8,20,25,37,40,41,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2000,Destiny's Child,Say My Name,4:31,Rock,1999-12-25T00:00:00.000+0000,2000-03-18T00:00:00.000+0000,83,83,44,38,16,13,16,16,16,18,17,14,1,1,1,2,2,3,5,5,5,7,10,13,14,18,23,23,34,37,43,47,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2000,"Iglesias, Enrique",Be With You,3:36,Latin,2000-04-01T00:00:00.000+0000,2000-06-24T00:00:00.000+0000,63,45,34,23,17,12,9,8,8,6,5,4,1,1,1,3,11,14,24,28,null,null,null,null,nul

* Un-pivot (transform columns into rows) the data, we can use the Spark SQL STACK statement - sub-query 'stacked_bb' Note we will only look at 20 weeks of the 75 odd weeks provided in this example
* We will derive a useful week_no that we can do calculations on 'x4th_week' is not helpful! Here we will use a Regular Expression, the most versitle string matching tool there is, more here https://www.rexegg.com/regex-quickstart.html 
* It might also be useful to derive the actual date relating to the week the track was at the rank recorded 
* Finally, we filter out any track that did not make it the top 100 in a given week

In [14]:
%sql
WITH stacked_bb AS (
  SELECT 
    year	
  , artist_inverted
  , track	
  , time	
  , genre	
  , date_entered	
  , date_peaked
  , STACK(20
    , 'x1st_week', x1st_week
    , 'x2nd_week', x2nd_week
    , 'x3rd_week', x3rd_week
    , 'x4th_week', x4th_week
    , 'x5th_week', x5th_week
    , 'x6th_week', x6th_week
    , 'x7th_week', x7th_week
    , 'x8th_week', x8th_week
    , 'x9th_week', x9th_week
    , 'x10th_week', x10th_week
    , 'x11th_week', x11th_week
    , 'x12th_week', x12th_week
    , 'x13th_week', x13th_week
    , 'x14th_week', x14th_week
    , 'x15th_week', x15th_week
    , 'x16th_week', x16th_week
    , 'x17th_week', x17th_week
    , 'x18th_week', x18th_week
    , 'x19th_week', x19th_week
    , 'x20th_week', x20th_week
    ) AS (
      week_no, rank
    )
  FROM billboard
)
SELECT
  year	
, artist_inverted	
, track	
, time	
, genre	
, date_entered
, date_peaked
, REGEXP_EXTRACT(week_no, '(\\d+)') AS week_no -- Regular expression to extract the number of week from the week_no string
, DATE_ADD(date_entered, (REGEXP_EXTRACT(week_no, '(\\d+)')*7)) AS date_at_rank -- Date at rank in question derived from date_entered + no weeks
, rank	
FROM stacked_bb
WHERE 
  rank IS NOT NULL -- Filter out NULL ranks (>100)




year,artist_inverted,track,time,genre,date_entered,date_peaked,week_no,date_at_rank,rank
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,1,2000-09-30,78
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,2,2000-10-07,63
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,3,2000-10-14,49
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,4,2000-10-21,33
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,5,2000-10-28,23
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,6,2000-11-04,15
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,7,2000-11-11,7
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,8,2000-11-18,5
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,9,2000-11-25,1
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,10,2000-12-02,1


Save our query above as a new dataframe so we can do some additional operations later on

In [16]:
sql_text = """
WITH stacked_bb AS (
  SELECT 
    year	
  , artist_inverted
  , track	
  , time	
  , genre	
  , date_entered	
  , date_peaked
  , STACK(20
    , 'x1st_week', x1st_week
    , 'x2nd_week', x2nd_week
    , 'x3rd_week', x3rd_week
    , 'x4th_week', x4th_week
    , 'x5th_week', x5th_week
    , 'x6th_week', x6th_week
    , 'x7th_week', x7th_week
    , 'x8th_week', x8th_week
    , 'x9th_week', x9th_week
    , 'x10th_week', x10th_week
    , 'x11th_week', x11th_week
    , 'x12th_week', x12th_week
    , 'x13th_week', x13th_week
    , 'x14th_week', x14th_week
    , 'x15th_week', x15th_week
    , 'x16th_week', x16th_week
    , 'x17th_week', x17th_week
    , 'x18th_week', x18th_week
    , 'x19th_week', x19th_week
    , 'x20th_week', x20th_week
    ) AS (
      week_no, rank
    )
  FROM billboard
)
SELECT
  year	
, artist_inverted	
, track	
, time	
, genre	
, date_entered
, date_peaked
, REGEXP_EXTRACT(week_no, '(\\\d+)') AS week_no
, DATE_ADD(date_entered, (REGEXP_EXTRACT(week_no, '(\\\d+)')*7)) AS date_at_rank
, rank	
FROM stacked_bb
WHERE 
  rank IS NOT NULL
"""

print(sql_text)

df_bb_tidy = spark.sql(sql_text)
df_bb_tidy.createOrReplaceTempView('billboard_tidy')

WITH stacked_bb AS (
 SELECT 
 year	
 , artist_inverted
 , track	
 , time	
 , genre	
 , date_entered	
 , date_peaked
 , STACK(50
 , 'x1st_week', x1st_week
 , 'x2nd_week', x2nd_week
 , 'x3rd_week', x3rd_week
 , 'x4th_week', x4th_week
 , 'x5th_week', x5th_week
 , 'x6th_week', x6th_week
 , 'x7th_week', x7th_week
 , 'x8th_week', x8th_week
 , 'x9th_week', x9th_week
 , 'x10th_week', x10th_week
 , 'x11th_week', x11th_week
 , 'x12th_week', x12th_week
 , 'x13th_week', x13th_week
 , 'x14th_week', x14th_week
 , 'x15th_week', x15th_week
 , 'x16th_week', x16th_week
 , 'x17th_week', x17th_week
 , 'x18th_week', x18th_week
 , 'x19th_week', x19th_week
 , 'x20th_week', x20th_week
 , 'x21st_week', x21st_week
 , 'x22nd_week', x22nd_week
 , 'x23rd_week', x23rd_week
 , 'x24th_week', x24th_week
 , 'x25th_week', x25th_week
 , 'x26th_week', x26th_week
 , 'x27th_week', x27th_week
 , 'x28th_week', x28th_week
 , 'x29th_week', x29th_week
 , 'x30th_week', x30th_week
 , 'x31st_week', x31st_week
 , 'x32nd_week', x32nd_week
 , 'x33rd_week', x33rd_week
 , 'x34th_week', x34th_week
 , 'x35th_week', x35th_week
 , 'x36th_week', x36th_week
 , 'x37th_week', x37th_week
 , 'x38th_week', x38th_week
 , 'x39th_week', x39th_week
 , 'x40th_week', x40th_week
 , 'x41st_week', x41st_week
 , 'x42nd_week', x42nd_week
 , 'x43rd_week', x43rd_week
 , 'x44th_week', x44th_week
 , 'x45th_week', x45th_week
 , 'x46th_week', x46th_week
 , 'x47th_week', x47th_week
 , 'x48th_week', x48th_week
 , 'x49th_week', x49th_week
 , 'x50th_week', x50th_week
 ) AS (
 week_no, rank
 )
 FROM billboard
)
SELECT
 year	
, artist_inverted	
, track	
, time	
, genre	
, date_entered
, date_peaked
, REGEXP_EXTRACT(week_no, '(\\d+)') AS week_no
, DATE_ADD(date_entered, (REGEXP_EXTRACT(week_no, '(\\d+)')*7)) AS date_at_rank
, rank	
FROM stacked_bb
WHERE 
 rank IS NOT NULL

In [17]:
%sql
SELECT * FROM billboard_tidy

year,artist_inverted,track,time,genre,date_entered,date_peaked,week_no,date_at_rank,rank
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,1,2000-09-30,78
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,2,2000-10-07,63
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,3,2000-10-14,49
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,4,2000-10-21,33
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,5,2000-10-28,23
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,6,2000-11-04,15
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,7,2000-11-11,7
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,8,2000-11-18,5
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,9,2000-11-25,1
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,10,2000-12-02,1


At this point the data are arguably tidy, but if we wanted to create a model for ths data that could be more easily extended and reused by other analysts we may consider spliting this dataset into two tables; one containing the track data and another containing the rank and date data. 

This type of 'data modeling' activity was more necessary in the days of relational databases (Boyce-Codd normal forms) and are somewhat less necessary in world of big data.

In any case, we cover splitting this data here as it may be useful in some circumstances

__Problems:__ 
* Multiple observational units (the `track` and its `rank`) in a single table.  

We’ll first create a table which contains the details of each track.  
__Note:__ key to this excercise is identifying a 'primary key': column(s) that uniquily identify a row (track) in this case. Here we are also creating our own convienient primary key `track_id` derived from the key cplumns `artist_inverted` and `track` using the windowing function `ROW_NUMBER()`

In [19]:
%sql
WITH track AS (
  SELECT DISTINCT 
    year
  , artist_inverted
  , track
  , time
  , genre 
  FROM billboard_tidy
)
SELECT 
  ROW_NUMBER() OVER (ORDER BY artist_inverted, track) AS track_id 
, year
, artist_inverted
, track
, time
, genre
FROM 
  song

track_id,year,artist_inverted,track,time,genre
1,2000,"""Elliott, Missy """"Misdemeanor""""""",Hot Boyz,3:51,Rap
2,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap
3,2000,2Ge+her,The Hardest Part Of Breaking Up (Is Getting Back Your Stuff),3:15,R&B
4,2000,3 Doors Down,Kryptonite,3:53,Rock
5,2000,3 Doors Down,Loser,4:24,Rock
6,2000,504 Boyz,Wobble Wobble,3:35,Rap
7,2000,98�,Give Me Just One Night (Una Noche),3:24,Rock
8,2000,A*Teens,Dancing Queen,3:44,Pop
9,2000,Aaliyah,I Don't Wanna,4:15,Rock
10,2000,Aaliyah,Try Again,4:03,Rock


In [20]:
sql_text = """
WITH track AS (
  SELECT DISTINCT 
    year
  , artist_inverted
  , track
  , time
  , genre 
  FROM billboard_tidy
)
SELECT 
  ROW_NUMBER() OVER (ORDER BY artist_inverted, track) AS track_id
, year
, artist_inverted
, track
, time
, genre
FROM 
  song
"""

df_track = spark.sql(sql_text)
df_track.createOrReplaceTempView('track')

Now create a ranks dataframe which only contains the `track_id`, `date` and the `rank` that may easily be joined back to the track table via `track_id`

In [22]:
sql_text = """
SELECT 
  t1.track_id, t2.date_at_rank, t2.rank
FROM 
  track t1
INNER JOIN 
  billboard_tidy t2
ON  t1.artist_inverted = t2.artist_inverted
AND t1.track = t2.track
ORDER BY track_id, date_at_rank
"""

df_rank = spark.sql(sql_text)
df_rank.createOrReplaceTempView('rank')
display(df_rank)

track_id,date_at_rank,rank
1,1999-12-04,36
1,1999-12-11,21
1,1999-12-18,13
1,1999-12-25,9
1,2000-01-01,7
1,2000-01-08,7
1,2000-01-15,5
1,2000-01-22,7
1,2000-01-29,7
1,2000-02-05,7


##2) Multiple variables stored in one column 
Tubercolosis Records from World Health Organization
This dataset documents the count of confirmed tuberculosis cases by country, year, age group and sex.

__Problems:__

* Some columns contain multiple values: sex and age.
* Mixture of zeros and missing values NaN. This is due to the data collection process and the distinction is important for this dataset.

In [24]:
df_tb_raw = spark.read.format('csv'
  ).option('header', 'true'
  ).option('inferSchema','true'
  ).option('delimiter',','
  ).load(S3_PREFIX + 'sagacity-analyst-scratch/workshops/tidy_data/data/tb-raw.csv')

df_tb_raw.createOrReplaceTempView('tb_raw')
display(df_tb_raw)

country,year,m_0-14,m_15-24,m_25-34,m_35-44,m_45-54,m_55-64,m65+,m_UKNOWN,f_0-14
AD,2000,0,0,1,0,0,0,0,null,null
AE,2000,2,4,4,6,5,12,10,null,3
AF,2000,52,228,183,149,129,94,80,null,93
AG,2000,0,0,0,0,0,0,1,null,1
AL,2000,2,19,21,14,24,19,16,null,3
AM,2000,2,152,130,131,63,26,21,null,1
AN,2000,0,0,1,2,0,0,0,null,0
AO,2000,186,999,1003,912,482,312,194,null,247
AR,2000,97,278,594,402,419,368,330,null,121
AS,2000,null,null,null,null,1,1,null,null,null


In order to tidy this dataset, we need to remove the different values from the header and unpivot them into rows. We’ll first need to `STACK` the `gender` + `age group` columns into a single column.  
Once we have that single column, we’ll derive three columns from this: `gender`, `age_lower` and `age_upper`. With those, we’ll be able to properly build a tidy dataset.

In [26]:
%sql
WITH stacked_tb AS (
  SELECT 
    country	
  , year
  , STACK(9
    , 'm_0-14', `m_0-14`	
    , 'm_15-24', `m_15-24`
    , 'm_25-34', `m_25-34`
    , 'm_35-44', `m_35-44`
    , 'm_45-54', `m_45-54` 	
    , 'm_55-64', `m_55-64`
    , 'm_65+', `m65+`
    , 'm_UKNOWN', CAST(m_UKNOWN AS INT)	
    , 'f_0-14', `f_0-14`
    ) AS (gender_age_band, cases)
  FROM 
    tb_raw
)
SELECT
  country	
, year
, gender_age_band
, REGEXP_EXTRACT(gender_age_band, '^(.+?)_')    AS gender -- extract all chars upto first instance of '_'
, REGEXP_EXTRACT(gender_age_band, '(\\d+)')     AS age_lower -- extract first instance of numbers found in string
, REGEXP_EXTRACT(gender_age_band, '[.-](\\d+)') AS age_upper -- extract numbers found in string after -
, NVL(cases, 0) AS cases -- NULL's to 0 
FROM
  stacked_tb
  

country,year,gender_age_band,gender,age_lower,age_upper,cases
AD,2000,m_0-14,m,0,14,0
AD,2000,m_15-24,m,15,24,0
AD,2000,m_25-34,m,25,34,1
AD,2000,m_35-44,m,35,44,0
AD,2000,m_45-54,m,45,54,0
AD,2000,m_55-64,m,55,64,0
AD,2000,m_65+,m,65,,0
AD,2000,m_UKNOWN,m,,,0
AD,2000,f_0-14,f,0,14,0
AE,2000,m_0-14,m,0,14,2


## 3) Variables are stored in both rows and columns
Global Historical Climatology Network Dataset
This dataset represents the daily weather records for a weather station (MX17004) in Mexico for five months in 2010.

__Problems:__
* Variables are stored in both rows (`tmin`, `tmax`) and columns (`days`).

In [28]:
df_weather_raw = spark.read.format('csv'
  ).option('header', 'true'
  ).option('inferSchema','true'
  ).option('delimiter',','
  ).load(S3_PREFIX + 'sagacity-analyst-scratch/workshops/tidy_data/data/weather-raw.csv')

df_weather_raw.createOrReplaceTempView('weather_raw')
display(df_weather_raw)

id,year,month,element,d1,d2,d3,d4,d5,d6,d7,d8
MX17004,2010,1,tmax,null,null,null,null,null,null,null,null
MX17004,2010,1,tmin,null,null,null,null,null,null,null,null
MX17004,2010,2,tmax,null,27.3,24.1,null,null,null,null,null
MX17004,2010,2,tmin,null,14.4,14.4,null,null,null,null,null
MX17004,2010,3,tmax,null,null,null,null,32.1,null,null,null
MX17004,2010,3,tmin,null,null,null,null,14.2,null,null,null
MX17004,2010,4,tmax,null,null,null,null,null,null,null,null
MX17004,2010,4,tmin,null,null,null,null,null,null,null,null
MX17004,2010,5,tmax,null,null,null,null,null,null,null,null
MX17004,2010,5,tmin,null,null,null,null,null,null,null,null


In [29]:
%sql
WITH weather_stacked AS (
  SELECT
    TRIM(id) AS id
  , year
  , month
  , element
  , STACK(8
    , 'd1', CAST(d1 AS DOUBLE) 
    , 'd2', CAST(d2 AS DOUBLE) 
    , 'd3', CAST(d3 AS DOUBLE) 
    , 'd4', CAST(d4 AS DOUBLE) 
    , 'd5', CAST(d5 AS DOUBLE) 
    , 'd6', CAST(d6 AS DOUBLE) 
    , 'd7', CAST(d7 AS DOUBLE) 
    , 'd8', CAST(d8 AS DOUBLE) 
    ) AS (day, temperature)
  FROM
    weather_raw
)
SELECT
  id
, TO_DATE(CONCAT(
    year, '-'
   , LPAD(month, 2, '0'), '-'
   , LPAD(REGEXP_EXTRACT(day, '(\\d+)'), 2, '0')
  )) AS date -- derive date extracting day number and combining with month & year
, element
, temperature
FROM
  weather_stacked

id,date,element,temperature
MX17004,2010-01-01,tmax,null
MX17004,2010-01-02,tmax,null
MX17004,2010-01-03,tmax,null
MX17004,2010-01-04,tmax,null
MX17004,2010-01-05,tmax,null
MX17004,2010-01-06,tmax,null
MX17004,2010-01-07,tmax,null
MX17004,2010-01-08,tmax,null
MX17004,2010-01-01,tmin,null
MX17004,2010-01-02,tmin,null


In [30]:
sql_text = """
WITH weather_stacked AS (
  SELECT
    TRIM(id) AS id
  , year
  , month
  , element
  , STACK(8
    , 'd1', CAST(d1 AS DOUBLE) 
    , 'd2', CAST(d2 AS DOUBLE) 
    , 'd3', CAST(d3 AS DOUBLE) 
    , 'd4', CAST(d4 AS DOUBLE) 
    , 'd5', CAST(d5 AS DOUBLE) 
    , 'd6', CAST(d6 AS DOUBLE) 
    , 'd7', CAST(d7 AS DOUBLE) 
    , 'd8', CAST(d8 AS DOUBLE) 
    ) AS (day, temperature)
  FROM
    weather_raw
)
SELECT
  id
, TO_DATE(CONCAT(
    year, '-'
   , LPAD(month, 2, '0'), '-'
   , LPAD(REGEXP_EXTRACT(day, '(\\\d+)'), 2, '0')
  )) AS date -- derive date extracting day number and combining with month & year
, element
, temperature
FROM
  weather_stacked
"""

df_weather_date = spark.sql(sql_text)
df_weather_date.createOrReplaceTempView('weather_date')
display(df_weather_date)

id,date,element,temperature
MX17004,2010-01-01,tmax,null
MX17004,2010-01-02,tmax,null
MX17004,2010-01-03,tmax,null
MX17004,2010-01-04,tmax,null
MX17004,2010-01-05,tmax,null
MX17004,2010-01-06,tmax,null
MX17004,2010-01-07,tmax,null
MX17004,2010-01-08,tmax,null
MX17004,2010-01-01,tmin,null
MX17004,2010-01-02,tmin,null


The `tmin` and `tmax` variables are in the single column, for analysis life will be easier if we show the min and max temperatures in two columns `min_temp` and `max_temp`  

For this we can use `PIVOT` to transform rows to columns (the opposite of `STACK`)

In [32]:
%sql
SELECT 
  id
, date
FROM weather_date PIVOT (
 MIN(temperature) -- note: we aren't really aggreating temperature here, nevertheless PIVOT requires we perform one (there is only one row per id and date so MIN, MAX, AVG, SUM are all equivalent here) 
 FOR element IN (
   'tmax' max_temp
 , 'tmin' min_temp
 )
)
WHERE 
  NOT (max_temp IS NULL AND min_temp IS NULL) -- remove rows with NULL's in min_temp or max_temp

id,date
MX17004,2010-02-03
MX17004,2010-03-05
MX17004,2010-02-02


The equivalent code using the Python API below.  
__Note:__ The implementation of `PIVOT` in Python is a little more flexible as it is not necessary to enumerate every element that might appear in the column we want to pivot on

In [34]:
from pyspark.sql.functions import min

display(
  df_weather_date.groupBy(
    'id'
  , 'date'
  ).pivot(
    'element'
  ).agg(
    min('temperature')
  ).dropna() # reomve rows with NULL's
)

id,date,tmax,tmin
MX17004,2010-02-03,24.1,14.4
MX17004,2010-03-05,32.1,14.2
MX17004,2010-02-02,27.3,14.4


## 4) One type in multiple tables
Dataset: Illinois Male Baby Names for the year 2014/2015.

__Problems:__

* The data is spread across multiple tables/files.  
* The `Year` variable is present in the file name.  

In order to load those different files into a single DataFrame, we can run a custom script that will append the files together. Furthermore, we’ll need to extract the “Year” variable from the file name.

In [36]:
# We can automattically load all the files into one dataframe using a file mask with a wildcard '*' 
# i.e. all files in 'sagacity-analyst-scratch/workshops/tidy_data/data/' that match the pattern '20*-baby-names-illinois.csv'
df_bnames_raw = spark.read.format('csv'
  ).option('header', 'true'
  ).option('inferSchema', 'true'
  ).option('delimiter', ','
  ).load(S3_PREFIX + 'sagacity-analyst-scratch/workshops/tidy_data/data/20*-baby-names-illinois.csv') 

df_bnames_raw.createOrReplaceTempView('bnames_raw')
display(df_bnames_raw)

rank,name,frequency,sex
1,Noah,837,Male
2,Alexander,747,Male
3,William,687,Male
4,Michael,680,Male
5,Liam,670,Male
6,Jacob,654,Male
7,Benjamin,649,Male
8,Mason,604,Male
9,Daniel,593,Male
10,Logan,593,Male


While the above is useful we need to extract part of the file name as a varible in a column for our dataframe. Something more complex is required ...

Below we will read each file individually, if the file matches our file mask we load it and append the `year` from the file name into a new column in the dataframe

__WARNING__ This example is getting a little more Python heavy but work through it - hopefully it will make sense!

In [38]:
import s3fs # Python library for file access on AWS S3
import re   # Python library for regular expressions
from pyspark.sql.functions import lit

# intially we have placeholder varible for our dataframe - we'll populate this with our data below
df_bnames = None

# initalise the filesystem for S3
fs = s3fs.S3FileSystem(key=dbutils.secrets.get("cstephenson", "access_key"), secret=dbutils.secrets.get("cstephenson", "secret_key").replace("/", "%2F"))

# get a list of the files in sagacity-analyst-scratch/workshops/tidy_data/data/
files =  fs.ls('sagacity-analyst-scratch/workshops/tidy_data/data/')
for file_name in files:
  
  # loop through all the files in sagacity-analyst-scratch/workshops/tidy_data/data/
  # if the file matches our file mask (using a regular expression), load it into a dataframe
  match = re.search(".*-baby-names-illinois.csv$", file_name)
  if match is not None:
    df_bnames_raw = spark.read.format('csv'
      ).option('header', 'true'
      ).option('inferSchema', 'true'
      ).option('delimiter', ','
      ).load(S3_PREFIX + match.string)
    
    # get the year from the file name, re.findall() returns a list of all sequences of numbers 
    # found in the file name, the year will be the first element here
    year = re.findall(r'(\d+)', file_name)
    
    # add the year variable as a new column in our dataframe named 'year'
    df_bnames_year = df_bnames_raw.withColumn('year', lit(year[0]))
    
    # if the df_bnames is empty (i.e. the first file we've loaded) initalise with the first dataframe `df_bnames_year`
    # else append the new dataframe/year to the dataframe we've already got
    if df_bnames is None:
        df_bnames = df_bnames_year
    else:
      df_bnames = df_bnames.unionByName(df_bnames_year)
    
# show the output
display(df_bnames)

rank,name,frequency,sex,year
1,Noah,837,Male,2014
2,Alexander,747,Male,2014
3,William,687,Male,2014
4,Michael,680,Male,2014
5,Liam,670,Male,2014
6,Jacob,654,Male,2014
7,Benjamin,649,Male,2014
8,Mason,604,Male,2014
9,Daniel,593,Male,2014
10,Logan,593,Male,2014


## Appendix

#### I) Dynamically building SQL for use with the `STACK` function
Previously we only STACKed 20 weeks of billboard data (mainly because it was too tideous to type!).  

But consider... suppose next week the client sends us an updated dataset with the 77th week column appended!  

We would hardly want to go back and have to edit our script again every single week.  

Let's automate this using some Python...

In [40]:
# Load the dataframe as normal 
df_billboard = spark.read.format('csv'
  ).option('header', 'true'
  ).option('inferSchema','true'
  ).option('delimiter',','
  ).load(S3_PREFIX + 'sagacity-analyst-scratch/workshops/tidy_data/data/billboard.csv')

df_billboard.createOrReplaceTempView('billboard')
display(df_billboard)

year,artist_inverted,track,time,genre,date_entered,date_peaked,x1st_week,x2nd_week,x3rd_week,x4th_week,x5th_week,x6th_week,x7th_week,x8th_week,x9th_week,x10th_week,x11th_week,x12th_week,x13th_week,x14th_week,x15th_week,x16th_week,x17th_week,x18th_week,x19th_week,x20th_week,x21st_week,x22nd_week,x23rd_week,x24th_week,x25th_week,x26th_week,x27th_week,x28th_week,x29th_week,x30th_week,x31st_week,x32nd_week,x33rd_week,x34th_week,x35th_week,x36th_week,x37th_week,x38th_week,x39th_week,x40th_week,x41st_week,x42nd_week,x43rd_week,x44th_week,x45th_week,x46th_week,x47th_week,x48th_week,x49th_week,x50th_week,x51st_week,x52nd_week,x53rd_week,x54th_week,x55th_week,x56th_week,x57th_week,x58th_week,x59th_week,x60th_week,x61st_week,x62nd_week,x63rd_week,x64th_week,x65th_week,x66th_week,x67th_week,x68th_week,x69th_week,x70th_week,x71st_week,x72nd_week,x73rd_week,x74th_week,x75th_week,x76th_week
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,78,63,49,33,23,15,7,5,1,1,1,1,1,1,1,1,1,1,1,2,3,7,10,12,15,22,29,31,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12T00:00:00.000+0000,2000-04-08T00:00:00.000+0000,15,8,6,5,2,3,2,2,1,1,1,1,1,1,1,1,1,1,8,15,19,21,26,36,48,47,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23T00:00:00.000+0000,2000-01-29T00:00:00.000+0000,71,48,43,31,20,13,7,6,4,4,4,6,4,2,1,1,1,2,1,2,4,8,8,12,14,17,21,24,30,34,37,46,47,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2000,Madonna,Music,3:45,Rock,2000-08-12T00:00:00.000+0000,2000-09-16T00:00:00.000+0000,41,23,18,14,2,1,1,1,1,2,2,2,2,2,4,8,11,16,20,25,27,27,29,44,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05T00:00:00.000+0000,2000-10-14T00:00:00.000+0000,57,47,45,29,23,18,11,9,9,11,1,1,1,1,4,8,12,22,23,43,44,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2000,Janet,Doesn't Really Matter,4:17,Rock,2000-06-17T00:00:00.000+0000,2000-08-26T00:00:00.000+0000,59,52,43,30,29,22,15,10,10,5,1,1,1,2,2,3,3,7,8,20,25,37,40,41,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2000,Destiny's Child,Say My Name,4:31,Rock,1999-12-25T00:00:00.000+0000,2000-03-18T00:00:00.000+0000,83,83,44,38,16,13,16,16,16,18,17,14,1,1,1,2,2,3,5,5,5,7,10,13,14,18,23,23,34,37,43,47,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2000,"Iglesias, Enrique",Be With You,3:36,Latin,2000-04-01T00:00:00.000+0000,2000-06-24T00:00:00.000+0000,63,45,34,23,17,12,9,8,8,6,5,4,1,1,1,3,11,14,24,28,null,null,null,null,nul

We will try to build a query __dynamically__ so that can `STACK` the week columns no matter how many weeks are preseent in the dataset

In [42]:
# intialise variables
no_weeks = 0
stack_cols = ""

# get the is of all columns in the dataframe - dataframe.columns returns a list of all columns
# in the dataframe, we can loop through each column
for col_name in df_billboard.columns:
  # if the column name contains the string '_week', build it into our STACK expression
  if '_week' in col_name:
    no_weeks += 1
    stack_cols += ", '" + col_name + "', CAST(" + col_name + " AS INT)\n" 
    
# assemble the STACK expression
sql_stack_expr = "STACK(" + str(no_weeks) + stack_cols + ") AS (week_no, rank)"

print(sql_stack_expr)

# build the SQL string with our dynamically constructed STACK expression
sql_text = """
WITH stacked_bb AS (
  SELECT 
    year	
  , artist_inverted
  , track	
  , time	
  , genre	
  , date_entered	
  , date_peaked
  , """ + sql_stack_expr + """
  FROM billboard
)
SELECT
  year	
, artist_inverted	
, track	
, time	
, genre	
, date_entered
, date_peaked
, REGEXP_EXTRACT(week_no, '(\\\d+)') AS week_no
, DATE_ADD(date_entered, (REGEXP_EXTRACT(week_no, '(\\\d+)')*7)) AS date_at_rank
, rank	
FROM stacked_bb
WHERE 
  rank IS NOT NULL
"""

print(sql_text)

# run the SQL and display the results
df_bb_tidy = spark.sql(sql_text)
display(df_bb_tidy)

year,artist_inverted,track,time,genre,date_entered,date_peaked,week_no,date_at_rank,rank
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,1,2000-09-30,78
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,2,2000-10-07,63
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,3,2000-10-14,49
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,4,2000-10-21,33
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,5,2000-10-28,23
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,6,2000-11-04,15
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,7,2000-11-11,7
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,8,2000-11-18,5
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,9,2000-11-25,1
2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23T00:00:00.000+0000,2000-11-18T00:00:00.000+0000,10,2000-12-02,1
